In [1]:
import os
import fnmatch
import numpy as np
import tensorflow as tf
import pandas as pd
import torch

In [2]:
os.chdir('/content/drive/MyDrive/07project')

In [3]:
def read_pdb_test(filename):
	
	with open(filename, 'r') as file:
		strline_L = file.readlines()
		# print(strline_L)

	X_list = list()
	Y_list = list()
	Z_list = list()
	atomtype_list = list()
	for strline in strline_L:
		# removes all whitespace at the start and end, including spaces, tabs, newlines and carriage returns
		stripped_line = strline.strip()
		# print(stripped_line)

		splitted_line = stripped_line.split('\t')
		
		X_list.append(float(splitted_line[0]))
		Y_list.append(float(splitted_line[1]))
		Z_list.append(float(splitted_line[2]))
		atomtype_list.append(str(splitted_line[3]))

	return X_list, Y_list, Z_list, atomtype_list

In [4]:
pro=[]
atom_num=[]
lig=[]
lig_num=[]
path_list = os.listdir('/content/drive/MyDrive/07project/testing_data_release/testing_data_release/testing_data')
path_list.sort(key=lambda x:str(x[:-9]))
for i in range(0,len(fnmatch.filter(os.listdir('/content/drive/MyDrive/07project/testing_data_release/testing_data_release/testing_data'), '*lig_cg.pdb'))):
  lig.append(read_pdb_test('/content/drive/MyDrive/07project/testing_data_release/testing_data_release/testing_data/'+path_list[2*i]))
  pro.append(read_pdb_test('/content/drive/MyDrive/07project/testing_data_release/testing_data_release/testing_data/'+path_list[2*i+1]))

In [5]:
#use the mean location of lig
def relocate_pro_lig_locate(lig,pro,width,new_lig,new_pro):
  for i in range(len(lig)):
    center = np.mean(lig[i][:3],axis=1)
    ligi=np.round(((lig[i][0]-center[0]).tolist(),(lig[i][1]-center[1]).tolist(),(lig[i][2]-center[2]).tolist()),3).tolist()
    ligi.append(lig[i][3])
    new_lig.append(ligi)
    
    proi=np.round(((pro[0]-center[0]).tolist(),(pro[1]-center[1]).tolist(),(pro[2]-center[2]).tolist()),3).tolist()
    proi.append(pro[3])
    new_pro.append(proi)

In [6]:
def choose_pro_lig(new_lig_pro,width):
  chosen_lig_pro = []
  for i in range(len(new_lig_pro)):
    chosen_lig_pro_i=[]
    for j in range(len(new_lig_pro[i][0])):
      if np.abs(new_lig_pro[i][0][j])<width and np.abs(new_lig_pro[i][1][j])<width and np.abs(new_lig_pro[i][2][j])<width:
        chosen_lig_pro_i.append([new_lig_pro[i][0][j],new_lig_pro[i][1][j],new_lig_pro[i][2][j],new_lig_pro[i][3][j]])
    chosen_lig_pro.append(chosen_lig_pro_i)
  return chosen_lig_pro

In [7]:
#def convert_4D(pro_lig_i,pro_lig_list,width,pro_or_lig):#for a protein
def convert_4D(pro_lig_i,pro_or_lig):#for a protein
    xyz_hp_4D = np.zeros((2*width, 2*width, 2*width, 1))
    for atom in pro_lig_i:
      (x, y, z, hp) = atom
      if hp =='h':
        if pro_or_lig==1: #pro-h=1
          xyz_hp_4D[int(x),int(y),int(z), 0] = 1
        if pro_or_lig==0: #lig-h=-1
          xyz_hp_4D[int(x),int(y),int(z), 0] = -1
      elif hp =='p':
        if pro_or_lig==1: #pro-p=2
          xyz_hp_4D[int(x),int(y),int(z), 0] = 2
        if pro_or_lig==0: #lig-p=-2
          xyz_hp_4D[int(x),int(y),int(z), 0] = -2
    return np.array(xyz_hp_4D)

In [8]:
def predict_for_one_protein(pro_i, lig,width, model):
    global current_lig_pro,test_pro,test_lig,test_pro_new,test_lig_new
    current_lig_pro = []
    test_pro = []
    test_lig = []
    test_pro_new=[]
    test_lig_new=[]
    relocate_pro_lig_locate(lig,pro_i,width,test_lig,test_pro)
    test_pro = choose_pro_lig(test_pro,width)
    test_lig = choose_pro_lig(test_lig,width)
    for i in range(len(test_lig)):
      cur_lig=convert_4D(test_lig[i],0)
      cur_pro=convert_4D(test_pro[i],1)
      test_lig_new.append(cur_lig)
      test_pro_new.append(cur_pro)
    current_lig_pro = np.concatenate([test_lig_new,test_pro_new],axis=-1)

    predict = model.predict(current_lig_pro)
    return predict

In [10]:
width=11
model=tf.keras.models.load_model('model_219new.h5')

In [11]:
a=predict_for_one_protein(pro[2], lig,width, model)

In [12]:
pd.DataFrame(a).sort_values(by=1,ascending=False).head(10)

0         1
21   0.028466  0.971534
656  0.033999  0.966001
22   0.068691  0.931309
195  0.068845  0.931155
757  0.082250  0.917750
598  0.099243  0.900757
564  0.134949  0.865051
194  0.221613  0.778387
125  0.237233  0.762767
337  0.285430  0.714570

In [ ]:
a=list(pd.DataFrame(a).sort_values(by=1,ascending=False).head(10).index)

In [13]:
X = np.zeros((824,11))
for i in range(len(pro)):
  if i % 10 == 0:
    print(i)
  X[i,0]=i+1
  predict_pro_i=predict_for_one_protein(pro[i], lig,width, model)
  top10=list(pd.DataFrame(predict_pro_i).sort_values(by=1,ascending=False).head(10).index)
  for j in range(len(top10)):
    top10[j]+=1
  X[i,1:]=top10

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820


In [14]:
Prediction = pd.DataFrame(X)

In [15]:
Prediction.columns=['pro_id','lig1_id','lig2_id','lig3_id','lig4_id','lig5_id','lig6_id','lig7_id','lig8_id','lig9_id','lig10_id']

In [16]:
Prediction=Prediction.astype(int)

In [17]:
Prediction

pro_id  lig1_id  lig2_id  lig3_id  ...  lig7_id  lig8_id  lig9_id  lig10_id
0         1       77      758      302  ...      746      439      232       747
1         2       77      302      439  ...      418      344      332       141
2         3       22      657       23  ...      565      195      126       338
3         4       34      758      302  ...      313      418      344       141
4         5       77      624      439  ...      344      141      384        34
..      ...      ...      ...      ...  ...      ...      ...      ...       ...
819     820      439      758      302  ...      635       18      418       344
820     821       77      758      302  ...      418      344      141       384
821     822      552      353      254  ...      302      439      243       505
822     823      410       77      442  ...      505      746      696       384
823     824       77      758      302  ...      418      344      141       384

[824 rows x 11 columns]

In [18]:
Prediction.to_csv('predictionNew.txt',index=False,sep='\t')